In [ ]:
from tweepy.streaming import StreamListener #class alows to listen based certain words
from tweepy import OAuthHandler #auth based on ID in other file
from tweepy import Stream #stream the tweets
from tweepy import API
from tweepy import Cursor

In [ ]:
class ListenerOutPut(StreamListener):#allows us to stream the tweets
    
    def on_data(self, data):#takes in data from streamlistener
        print(data)#prints the data
        return True
    
    def on_error(self, status):#tells us if an error happens
        if status == (420): #so if you cross the rate limit returns false on connection
            return false
        print(status)
        
    def __init__(self, filename_tweet):
        self.filename_tweet = filename_tweet #writes to a file dont have to view in console
        
        
        
#all of this part allows us to stream tweet made in real time to STDOUT

In [ ]:
class AuthenticateTwitter():
    def AuthenticTwitterApp(self):
        Authenticate = OAuthHandler("CONSUMER_KEY","CONSUMER _S_KEY ")
        Authenticate.set_access_token("ACCESS_TOKEN","ACCESS_S_TOKEN")
        return Authenticate

In [ ]:
class ClientTwitter():
    def __init__ (self):
        self.Authenticate = AuthenticateTwitter().AuthenticTwitterApp()
        self.twitter_client = API(self.Authenticate)
        
    def Tweet_get_user(self, num_tweets):
        My_Tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline).items(num_tweets):
            My_Tweets.append(tweet)
        return My_Tweets

In [ ]:
class Tweets():
    def __init__(self):
        self.authenticate_twitter = AuthenticateTwitter()
        
    def Streaming_Tweets(self, HT_List, filename_tweet):#allows us to write to a file and is overall more convinient 
        
        OPListener = ListenerOutPut(filename_tweet) #inherite
        
        ###Authenication###
        Authenticate = self.authenticate_twitter.AuthenticTwitterApp()
        
        AuthStream = Stream(Authenticate, OPListener)
        AuthStream.filter(track = HT_List)
#allows us to see what we want when it comes to seeing live tweets

In [ ]:
if __name__ == "__main__":
    HT_List = ["Dog", "Cat"]
    filename_tweet = "tweetStream.txt"
    twitter_client = ClientTwitter()
    print(twitter_client.Tweet_get_user(1))
    #Tstream = Tweets()
    #Tstream.Streaming_Tweets(HT_List, filename_tweet)
    
    